In [2]:
import pandas as pd
import sqlite3

In [3]:
playoff_20152016_data_query = ('''
SELECT game.game_id, game_plays.*, game_plays_players.player_id FROM game
JOIN game_plays ON game.game_id = game_plays.game_id
JOIN game_plays_players ON game_plays.play_id = game_plays_players.play_id
WHERE game.type = 'P'
    AND game.season = 20152016
    AND (
        game.home_team_id = 5
        OR
        game.away_team_id = 5
        );
''')

In [4]:
conn = sqlite3.connect('NHL_data.db')

In [5]:
pens_201516_playoff_df = pd.read_sql_query(playoff_20152016_data_query, conn)

In [6]:
pens_201516_playoff_df.head()

,game_id,play_id,game_id,team_id_for,team_id_against,event,secondaryType,x,y,period,periodType,periodTime,periodTimeRemaining,dateTime,goals_away,goals_home,description,st_x,st_y,player_id
0,2015030141,2015030141_10,2015030141,3.0,5.0,Hit,None,-96.0,31.0,1,REGULAR,102,1098.0,2016-04-14 01:12:35,0,0,Jesper Fast hit Olli Maatta,96.0,-31.0,8475855
1,2015030141,2015030141_10,2015030141,3.0,5.0,Hit,None,-96.0,31.0,1,REGULAR,102,1098.0,2016-04-14 01:12:35,0,0,Jesper Fast hit Olli Maatta,96.0,-31.0,8476874
2,2015030141,2015030141_100,2015030141,3.0,5.0,Hit,None,-98.0,30.0,1,REGULAR,978,222.0,2016-04-14 01:44:19,0,0,Jesper Fast hit Carl Hagelin,98.0,-30.0,8474176
3,2015030141,2015030141_100,2015030141,3.0,5.0,Hit,None,-98.0,30.0,1,REGULAR,978,222.0,2016-04-14 01:44:19,0,0,Jesper Fast hit Carl Hagelin,98.0,-30.0,8475855
4,2015030141,2015030141_101,2015030141,3.0,5.0,Hit,None,-89.0,-37.0,1,REGULAR,985,215.0,2016-04-14 01:44:26,0,0,Jesper Fast hit Justin Schultz,89.0,37.0,8474602


In [7]:
pens_201516_playoff_df.columns

Index(['game_id', 'play_id', 'game_id', 'team_id_for', 'team_id_against',
       'event', 'secondaryType', 'x', 'y', 'period', 'periodType',
       'periodTime', 'periodTimeRemaining', 'dateTime', 'goals_away',
       'goals_home', 'description', 'st_x', 'st_y', 'player_id'],
      dtype='object')

In [11]:
pens_201516_playoff_df.iloc[:,~pens_201516_playoff_df.columns.duplicated()]

,game_id,play_id,team_id_for,team_id_against,event,secondaryType,x,y,period,periodType,periodTime,periodTimeRemaining,dateTime,goals_away,goals_home,description,st_x,st_y,player_id
0,2015030141,2015030141_10,3.0,5.0,Hit,None,-96.0,31.0,1,REGULAR,102,1098.0,2016-04-14 01:12:35,0,0,Jesper Fast hit Olli Maatta,96.0,-31.0,8475855
1,2015030141,2015030141_10,3.0,5.0,Hit,None,-96.0,31.0,1,REGULAR,102,1098.0,2016-04-14 01:12:35,0,0,Jesper Fast hit Olli Maatta,96.0,-31.0,8476874
2,2015030141,2015030141_100,3.0,5.0,Hit,None,-98.0,30.0,1,REGULAR,978,222.0,2016-04-14 01:44:19,0,0,Jesper Fast hit Carl Hagelin,98.0,-30.0,8474176
3,2015030141,2015030141_100,3.0,5.0,Hit,None,-98.0,30.0,1,REGULAR,978,222.0,2016-04-14 01:44:19,0,0,Jesper Fast hit Carl Hagelin,98.0,-30.0,8475855
4,2015030141,2015030141_101,3.0,5.0,Hit,None,-89.0,-37.0,1,REGULAR,985,215.0,2016-04-14 01:44:26,0,0,Jesper Fast hit Justin Schultz,89.0,37.0,8474602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13174,2015030416,2015030416_97,5.0,28.0,Shot,Wrist Shot,-86.0,9.0,1,REGULAR,994,206.0,2016-06-13 01:49:21,1,0,Sidney Crosby Wrist Shot saved by Martin Jones,86.0,-9.0,8474889
13175,2015030416,2015030416_98,28.0,5.0,Hit,None,-97.0,28.0,1,REGULAR,998,202.0,2016-06-13 01:49:25,1,0,Paul Martin hit Sidney Crosby,-97.0,28.0,8468542
13176,2015030416,2015030416_98,28.0,5.0,Hit,None,-97.0,28.0,1,REGULAR,998,202.0,2016-06-13 01:49:25,1,0,Paul Martin hit Sidney Crosby,-97.0,28.0,8471675
13177,2015030416,2015030416_99,28.0,5.0,Blocked Shot,None,-73.0,0.0,1,REGULAR,1001,199.0,2016-06-13 01:51:38,1,0,Joel Ward blocked shot from Phil Kessel,-73.0,0.0,8468208


In [15]:
pens_201516_playoff_df.drop(['dateTime','st_x','st_y'], axis=1)

,game_id,play_id,game_id,team_id_for,team_id_against,event,secondaryType,x,y,period,periodType,periodTime,periodTimeRemaining,goals_away,goals_home,description,player_id
0,2015030141,2015030141_10,2015030141,3.0,5.0,Hit,None,-96.0,31.0,1,REGULAR,102,1098.0,0,0,Jesper Fast hit Olli Maatta,8475855
1,2015030141,2015030141_10,2015030141,3.0,5.0,Hit,None,-96.0,31.0,1,REGULAR,102,1098.0,0,0,Jesper Fast hit Olli Maatta,8476874
2,2015030141,2015030141_100,2015030141,3.0,5.0,Hit,None,-98.0,30.0,1,REGULAR,978,222.0,0,0,Jesper Fast hit Carl Hagelin,8474176
3,2015030141,2015030141_100,2015030141,3.0,5.0,Hit,None,-98.0,30.0,1,REGULAR,978,222.0,0,0,Jesper Fast hit Carl Hagelin,8475855
4,2015030141,2015030141_101,2015030141,3.0,5.0,Hit,None,-89.0,-37.0,1,REGULAR,985,215.0,0,0,Jesper Fast hit Justin Schultz,8474602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13174,2015030416,2015030416_97,2015030416,5.0,28.0,Shot,Wrist Shot,-86.0,9.0,1,REGULAR,994,206.0,1,0,Sidney Crosby Wrist Shot saved by Martin Jones,8474889
13175,2015030416,2015030416_98,2015030416,28.0,5.0,Hit,None,-97.0,28.0,1,REGULAR,998,202.0,1,0,Paul Martin hit Sidney Crosby,8468542
13176,2015030416,2015030416_98,2015030416,28.0,5.0,Hit,None,-97.0,28.0,1,REGULAR,998,202.0,1,0,Paul Martin hit Sidney Crosby,8471675
13177,2015030416,2015030416_99,2015030416,28.0,5.0,Blocked Shot,None,-73.0,0.0,1,REGULAR,1001,199.0,1,0,Joel Ward blocked shot from Phil Kessel,8468208


In [16]:
pens_201516_playoff_df.event.unique()

array(['Hit', 'Shot', 'Faceoff', 'Blocked Shot', 'Missed Shot',
       'Takeaway', 'Goal', 'Penalty', 'Giveaway'], dtype=object)

In [17]:
pens_201516_playoff_df.secondaryType.unique()

array([None, 'Wrist Shot', 'Snap Shot', 'Slap Shot', 'Interference',
       'Backhand', 'Holding', 'Tripping', 'Tip-In',
       'Hi stick - double minor', 'Roughing', 'Interference - Goalkeeper',
       'Wrap-around', 'Hooking', 'Illegal check to head',
       'Unsportsmanlike conduct', 'Slashing', 'Cross checking',
       'Fighting', 'Boarding', 'Holding the stick', 'Hi-sticking',
       'Embellishment', 'Too many men on the ice',
       'Delaying Game - Puck over glass', 'Elbowing', 'Misconduct',
       'Deflected'], dtype=object)

In [21]:
pens_201516_playoff_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13179 entries, 0 to 13178
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   game_id              13179 non-null  int64  
 1   play_id              13179 non-null  object 
 2   game_id              13179 non-null  int64  
 3   team_id_for          13179 non-null  float64
 4   team_id_against      13179 non-null  float64
 5   event                13179 non-null  object 
 6   secondaryType        3586 non-null   object 
 7   x                    13163 non-null  float64
 8   y                    13165 non-null  float64
 9   period               13179 non-null  int64  
 10  periodType           13179 non-null  object 
 11  periodTime           13179 non-null  int64  
 12  periodTimeRemaining  13179 non-null  float64
 13  dateTime             13179 non-null  object 
 14  goals_away           13179 non-null  int64  
 15  goals_home           13179 non-null 

In [24]:
secondary_type_percent_null = (pens_201516_playoff_df['secondaryType'].isnull().sum())/13179*100
secondary_type_percent_null

72.79004476819182